In [43]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.io as pio
from plotly.subplots import make_subplots
import talib as tb
import yfinance as yf


import datetime
from datetime import date
from datetime import datetime as dt
from dateutil.relativedelta import *
from datetime import timedelta


To comment several lines of code use CTRL + K + C / CTRL + K + U

In [44]:
def df_converter(df): 
    df_sp500 = yf.download('^GSPC', 
                      start='2017-11-09', 
                      end='2022-05-08', 
                      progress=False)
    df_dollar = yf.download('DX=F', 
                      start='2017-11-09', 
                      end='2022-05-08', 
                      progress=False)
    # clearing dollar and sp500 df
    df_dollar.drop(['Open', 'High', 'Low', 'Adj Close', 'Volume'], axis=1, inplace=True)
    df_dollar.rename(columns={"Close": "dollar_close"}, inplace=True)
    df_sp500.drop(['Open', 'High', 'Low', 'Adj Close', 'Volume'], axis=1, inplace=True)
    df_sp500.rename(columns={"Close": "sp500_close"}, inplace=True)
    # clearing general df
    #df_eth.drop('Unnamed: 0', axis=1, inplace=True)
    #df.drop('adj_close', axis=1, inplace=True)
    df.index = df.index.astype('datetime64[ns]')
    # MA df
    df_ma = df['Close'].to_frame()
    df_ma['SMA30'] = df_ma['Close'].rolling(15).mean()
    df_ma['CMA30'] = df_ma['Close'].expanding().mean()
    df_ma['EMA30'] = tb.EMA(df_ma['Close'], timeperiod=15)
    df_ma.dropna(inplace=True)
    # Stoch df
    slowk, slowd = tb.STOCH(df["High"], df["Low"], df["Close"], fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
    df_stoch = pd.DataFrame(index=df.index,
                                data={"slowk": slowk,
                                    "slowd": slowd})
    df_stoch.dropna(inplace=True)
    # for later use in the concat
    stoch_c = ['slowk', 'slowd']
    # MACD df 
    macd, macdsignal, macdhist = tb.MACD(df.Close, fastperiod=12, slowperiod=26, signalperiod=9)
    df_macd = pd.DataFrame(index=df.index,
                            data={"macd": macd,
                                  "macdsignal": macdsignal,
                                  "macdhist": macdhist})
    df_macd.dropna(inplace=True)
    # for later use in the concat
    macd_c = ['macd', 'macdsignal', 'macdhist']
    # bb df
    upper, middle, lower = tb.BBANDS(df["Close"], timeperiod=15)
    df_bands = pd.DataFrame(index=df.index,
                                data={"bb_low": lower,
                                    "bb_ma": middle,
                                    "bb_high": upper})
    df_bands.dropna(inplace=True)
    # for later use in the concat
    bands_c = ['bb_low', 'bb_ma', 'bb_high']
    # rsi df
    rsi = tb.RSI(df['Close'], timeperiod=15)
    df_rsi = pd.DataFrame(index=df.index,
                            data={"close": df['Close'],
                                  "rsi": rsi})

    df_rsi.dropna(inplace=True)
    #stdev df
    stdev = tb.STDDEV(df['Close'], timeperiod=15, nbdev=1)
    df_stdev = pd.DataFrame(index=df.index,
                            data={"close": df['Close'],
                                  "stdev": stdev})
    df_stdev.dropna(inplace=True)
    # adx df
    adx = tb.ADX(df['High'], df['Low'], df['Close'], timeperiod=15)
    df_adx = pd.DataFrame(index=df.index,
                                data={"close": df['Close'],
                                    "adx": adx})

    df_adx.dropna(inplace=True)

    # concat 
    result =pd.concat([df, df_ma[['SMA30','CMA30','EMA30']], df_adx['adx'], df_bands[bands_c], df_macd[macd_c], df_rsi['rsi'], df_stdev['stdev'], df_stoch[stoch_c], df_dollar['dollar_close'], df_sp500['sp500_close']], axis=1)
    result.fillna(method='ffill', inplace=True)
    result.dropna(inplace=True)

    return result 

In [45]:
df_btc = yf.download('BTC-USD',
                      end=date.today(), 
                      progress=False,
    )

In [46]:
df_btc = df_converter(df_btc)

In [47]:
df_btc.head()

,Open,High,Low,Close,Adj Close,Volume,SMA30,CMA30,EMA30,adx,...,bb_high,macd,macdsignal,macdhist,rsi,stdev,slowk,slowd,dollar_close,sp500_close
Date,,,,,,,,,,,,,,,,,,,,,
2017-11-09,7446.830078,7446.830078,7101.520020,7143.580078,7143.580078,3226249984,6720.195964,1012.489040,6798.973215,37.428679,...,7929.899976,547.298468,521.809901,25.488567,65.836431,604.852006,33.184032,38.772059,94.351997,2584.620117
2017-11-10,7173.729980,7312.000000,6436.870117,6618.140137,6618.140137,5208249856,6767.749967,1017.359285,6776.369081,36.238180,...,7899.045668,482.149729,513.877867,-31.728137,55.203600,565.647850,30.022892,33.577955,94.278000,2582.300049
2017-11-11,6618.609863,6873.149902,6204.220215,6357.600098,6357.600098,4908680192,6806.196647,1021.994910,6724.022958,34.548539,...,7835.311479,404.828866,492.068066,-87.239201,50.841315,514.557416,13.668599,25.625174,94.278000,2582.300049
2017-11-12,6295.450195,6625.049805,5519.009766,5950.069824,5950.069824,8957349888,6819.328646,1026.269043,6627.278816,32.896648,...,7798.154128,307.126870,455.079827,-147.952957,44.895661,489.412741,14.127749,19.273080,94.278000,2582.300049
2017-11-13,5938.250000,6811.189941,5844.290039,6559.490234,6559.490234,6263249920,6846.371322,1031.063862,6618.805243,30.978407,...,7771.077715,275.694434,419.202748,-143.508315,53.591318,462.353197,27.607649,18.467999,94.388000,2584.840088


In [48]:
df_sol = yf.download('SOL-USD',
                      end=date.today(), 
                      progress=False,
    )

In [49]:
df_btc['rsi'].max()

93.77913607014075

In [50]:
df_btc.columns.tolist()[6:]

['SMA30',
 'CMA30',
 'EMA30',
 'adx',
 'bb_low',
 'bb_ma',
 'bb_high',
 'macd',
 'macdsignal',
 'macdhist',
 'rsi',
 'stdev',
 'slowk',
 'slowd',
 'dollar_close',
 'sp500_close']

In [212]:
df_btc[df_btc.index == '2022-04-25']

,Open,High,Low,Close,Adj Close,Volume,SMA30,CMA30,EMA30,adx,...,bb_high,macd,macdsignal,macdhist,rsi,stdev,slowk,slowd,dollar_close,sp500_close
Date,,,,,,,,,,,,,,,,,,,,,
2022-04-25,39472.605469,40491.753906,38338.378906,40458.308594,40458.308594,35445730570,40322.10625,12030.588997,40763.991139,20.839138,...,41643.822748,-823.171966,-728.838232,-94.333734,45.335035,660.858249,19.255884,18.573572,101.768997,4296.120117


In [220]:
def candlestick(df, days, comparison = None, indicators = None):

    today = date.today()

    df = df[today - pd.offsets.Day(days):]
    df_normal = df.copy() # df with absolute values
    df= df.pct_change()

    color_palete= ['violet', 'magenta', 'turquoise', 'mediumorchid','lightpink', 'mediumpurple', 'mediumvioletred', 'darkviolet']

    #Editing the text(hover) on the candlestick to get the absolute value and the % changes   
    hovertext=[]
    for i in range(len(df_normal.Open)):
        hovertext.append('Open: '+str(df_normal.Open[i].round(2))+'<br>% change:' + str(df.Open[i].round(3))
                        +'<br>High: '+str(df_normal.High[i].round(2)) +'<br>% change:' + str(df.High[i].round(3))
                        +'<br>Low: '+str(df_normal.Low[i].round(2)) +'<br>% change:' + str(df.Low[i].round(3))
                        +'<br>Close: '+str(df_normal.Close[i].round(2)) +'<br>% change:' + str(df.Close[i].round(3)))

    if comparison is None and indicators is None: 

        fig = go.Figure(
            data = [
                go.Candlestick(
                    x = df_normal.index,
                    open = df_normal.Open,
                    high = df_normal.High,
                    low = df_normal.Low,
                    close = df_normal.Close,
                    text= hovertext,
                    hoverinfo='text'
                )
            ]
        )
        
        fig.update_layout(width=1200, height= 700)

    elif comparison is not None and indicators is None: 

        comparison = comparison[today - pd.offsets.Day(days):]
        comparison = comparison.pct_change()

        fig = go.Figure(
            data = [
                go.Candlestick(
                    x = df.index,
                    open = df.Open,
                    high = df.High,
                    low = df.Low,
                    close = df.Close,
                    text= hovertext,
                    hoverinfo='text'
                ),
                go.Scatter(
                    x = comparison.index, 
                    y = comparison.Close,
                    mode = 'lines', 
                    name = 'SOL',
                    line = {'color': 'orange'},
                    customdata=df_normal[['Open','High','Low','Close']],
                            hovertemplate='<br>'.join([
                                'Percentual Change: %{y}',
                                'Date: %{x}',
                                'Open: %{customdata[0]:.3f}',
                                'High: %{customdata[1]:.3f}',
                                'Low: %{customdata[2]:.3f}',
                                'Close: %{customdata[3]:.3f}'
                            ])
                )

            ],
            layout = go.Layout(yaxis=dict(tickformat=".2%"))
        )   

        fig.update_layout(width=1200, height= 700)

    elif comparison is None and indicators is not None:

        fig = make_subplots(rows=5, cols=1, specs = [[{ }], [{ }], [{ }], [{ }], [{ }]], vertical_spacing = 0.10, 
                            row_heights=[180, 30, 30, 30, 30])
        fig.update_yaxes(tickformat=".2%", row=1)
        fig.append_trace(
                        go.Candlestick(
                            x = df.index,
                            open = df.Open,
                            high = df.High,
                            low = df.Low,
                            close = df.Close,
                            text= hovertext,
                            hoverinfo='text'
                        ), row=1, col=1)
        
        for i,v in enumerate(indicators): 
            if v == 'rsi': 
                fig.append_trace(go.Scatter(
                        x=df_normal['rsi'].index,
                        y=df_normal['rsi'],
                        mode="lines",
                        name = 'rsi'
                        ),  row=3, col=1
                    )
                
                fig.add_annotation(xref="x domain",yref="y domain",x=0.5, y=1.5, showarrow=False,
                   text="<b>RSI</b>", row=3, col=1)
                
                fig.add_hline(y=30 , line_width=2, line_dash="dash", line_color="darkslategray", row=3, col=1)
                fig.add_hline(y=70, line_width=2, line_dash="dash", line_color="darkslategray", row=3, col=1)

            
            elif v == 'adx': 
                fig.append_trace(go.Scatter(
                        x=df_normal['adx'].index,
                        y=df_normal['adx'],
                        mode="lines",
                        name = 'adx'
                        ),  row=5, col=1
                    )
                
                fig.add_annotation(xref="x domain",yref="y domain",x=0.5, y=1.5, showarrow=False,
                   text="<b>ADX</b>", row=5, col=1)

            elif v == 'stochastic': 
                fig.append_trace(go.Scatter(
                        x=df_normal['slowk'].index,
                        y=df_normal['slowk'],
                        mode="lines",
                        name = 'slowk'
                        ),  row=2, col=1
                    )
                fig.add_trace(
                        go.Scatter(
                            x=df_normal['slowd'].index,
                            y=df_normal['slowd'],
                            mode="lines",
                            name = 'slowd'
                            ),  row=2, col=1
                    )

                fig.add_annotation(xref="x domain",yref="y domain",x=0.5, y=1.5, showarrow=False,
                   text="<b>Stochastic Oscilator</b>", row=2, col=1)
                fig.add_hline(y=20 , line_width=2, line_dash="dash", line_color="darkslategray", row=2, col=1)
                fig.add_hline(y=80, line_width=2, line_dash="dash", line_color="darkslategray", row=2, col=1)

            elif v == 'macd': 
                fig.append_trace(go.Scatter(
                        x=df_normal['macd'].index,
                        y=df_normal['macd'],
                        mode="lines",
                        name = 'macd'
                        ),  row=4, col=1
                    )
                fig.add_trace(
                        go.Scatter(
                            x=df_normal['macdsignal'].index,
                            y=df_normal['macdsignal'],
                            mode="lines",
                            name = 'macd signal'
                            ),  row=4, col=1
                    )
                
                fig.add_annotation(xref="x domain",yref="y domain",x=0.5, y=1.5, showarrow=False,
                   text="<b>MACD</b>", row=4, col=1)
                          
            elif v == 'bolinger bands':

                fig.add_trace(go.Scatter(x=df['bb_low'].index,
                    y=df['bb_low'],
                    fill=None,
                    mode='lines',
                    line_color='indigo',
                    name = 'bb_low',
                    customdata=df_normal[['Open','High','Low','Close']],
                            hovertemplate='<br>'.join([
                                'Percentual Change: %{y}',
                                'Date: %{x}',
                                'Open: %{customdata[0]:.3f}',
                                'High: %{customdata[1]:.3f}',
                                'Low: %{customdata[2]:.3f}',
                                'Close: %{customdata[3]:.3f}'
                            ])
                    
                    ))
                fig.add_trace(go.Scatter(
                    x=df['bb_high'].index,
                    y=df['bb_high'],
                    name = 'bb_high',
                    fill='tonexty', # fill area between trace0 and trace1
                    mode='lines', line_color='indigo', 
                    customdata=df_normal[['Open','High','Low','Close']],
                            hovertemplate='<br>'.join([
                                'Percentual Change: %{y}',
                                'Date: %{x}',
                                'Open: %{customdata[0]:.3f}',
                                'High: %{customdata[1]:.3f}',
                                'Low: %{customdata[2]:.3f}',
                                'Close: %{customdata[3]:.3f}'
                            ])))    

            else:
                for ind, value in enumerate(color_palete):
                    if i == ind: 
                        fig.add_trace(
                        go.Scatter(
                            x=df[v].index,
                            y=df[v],
                            mode="lines",
                            name = v,
                            line_color=value,
                            customdata=df_normal[['Open','High','Low','Close']],
                            hovertemplate='<br>'.join([
                                'Percentual Change: %{y}',
                                'Date: %{x}',
                                'Open: %{customdata[0]:.3f}',
                                'High: %{customdata[1]:.3f}',
                                'Low: %{customdata[2]:.3f}',
                                'Close: %{customdata[3]:.3f}'
                            ]),
                        )
                        )
                
        fig.update_layout(width=1300, height= 2000)
    
    else:

        comparison = comparison[today - pd.offsets.Day(days):]
        comparison = comparison.pct_change()
        
        fig = make_subplots(rows=5, cols=1, specs = [[{ }], [{ }], [{ }],  [{ }], [{ }]], vertical_spacing = 0.10, 
                            row_heights=[180, 30, 30, 30, 30] )
        fig.update_yaxes(tickformat=".2%", row = 1)

        fig.append_trace(
                go.Candlestick(
                    x = df.index,
                    open = df.Open,
                    high = df.High,
                    low = df.Low,
                    close = df.Close,
                    text= hovertext,
                    hoverinfo='text'
                ), row=1, col=1)
        
        
        
        fig.add_trace(go.Scatter(
                        x = comparison.index, 
                        y = comparison.Close,
                        mode = 'lines', 
                        name = 'SOL',
                        line = {'color': 'orange'},
                        customdata=df_normal[['Open','High','Low','Close']],
                            hovertemplate='<br>'.join([
                                'Percentual Change: %{y}',
                                'Date: %{x}',
                                'Open: %{customdata[0]:.3f}',
                                'High: %{customdata[1]:.3f}',
                                'Low: %{customdata[2]:.3f}',
                                'Close: %{customdata[3]:.3f}'
                            ])
                        ))

        for i,v in enumerate(indicators): 
            if v == 'rsi': 
            
                fig.append_trace(go.Scatter(
                        x=df_normal['rsi'].index,
                        y=df_normal['rsi'],
                        mode="lines",
                        name = 'rsi'
                        ),  row=3, col=1
                    )
                
                fig.add_annotation(xref="x domain",yref="y domain",x=0.5, y=1.5, showarrow=False,
                   text="<b>RSI</b>", row=3, col=1)
                fig.add_hline(y=30 , line_width=2, line_dash="dash", line_color="darkslategray", row=3, col=1)
                fig.add_hline(y=70, line_width=2, line_dash="dash", line_color="darkslategray", row=3, col=1)

            elif v == 'adx': 
                fig.append_trace(go.Scatter(
                        x=df_normal['adx'].index,
                        y=df_normal['adx'],
                        mode="lines",
                        name = 'adx'
                        ),  row=5, col=1
                    )
                
                fig.add_annotation(xref="x domain",yref="y domain",x=0.5, y=1.5, showarrow=False,
                   text="<b>ADX</b>", row=5, col=1)

            elif v == 'stochastic': 
                fig.append_trace(go.Scatter(
                        x=df_normal['slowk'].index,
                        y=df_normal['slowk'],
                        mode="lines",
                        name = 'slowk'
                        ),  row=2, col=1
                    )
                fig.add_trace(
                        go.Scatter(
                            x=df_normal['slowd'].index,
                            y=df_normal['slowd'],
                            mode="lines",
                            name = 'slowd'
                            ),  row=2, col=1
                    )

                fig.add_annotation(xref="x domain",yref="y domain",x=0.5, y=1.5, showarrow=False,
                   text="<b>Stochastic Oscilator</b>", row=2, col=1)
                fig.add_hline(y=20 , line_width=2, line_dash="dash", line_color="darkslategray", row=2, col=1)
                fig.add_hline(y=80, line_width=2, line_dash="dash", line_color="darkslategray", row=2, col=1)

            elif v == 'macd': 
                fig.append_trace(go.Scatter(
                        x=df_normal['macd'].index,
                        y=df_normal['macd'],
                        mode="lines",
                        name = 'macd'
                        ),  row=4, col=1
                    )
                fig.add_trace(
                        go.Scatter(
                            x=df_normal['macdsignal'].index,
                            y=df_normal['macdsignal'],
                            mode="lines",
                            name = 'macd signal'
                            ),  row=4, col=1
                    )
                
                fig.add_annotation(xref="x domain",yref="y domain",x=0.5, y=1.5, showarrow=False,
                   text="<b>MACD</b>", row=4, col=1)
            
            elif v == 'bolinger bands':

                fig.add_trace(go.Scatter(x=df['bb_low'].index,
                    y=df['bb_low'],
                    fill=None,
                    mode='lines',
                    line_color='indigo',
                    name = 'bb_low',
                    customdata=df_normal[['Open','High','Low','Close']],
                            hovertemplate='<br>'.join([
                                'Percentual Change: %{y}',
                                'Date: %{x}',
                                'Open: %{customdata[0]:.3f}',
                                'High: %{customdata[1]:.3f}',
                                'Low: %{customdata[2]:.3f}',
                                'Close: %{customdata[3]:.3f}'
                            ])
                    
                    ))
                fig.add_trace(go.Scatter(
                    x=df['bb_high'].index,
                    y=df['bb_high'],
                    name = 'bb_high',
                    fill='tonexty', # fill area between trace0 and trace1
                    mode='lines', line_color='indigo', 
                    customdata=df_normal[['Open','High','Low','Close']],
                            hovertemplate='<br>'.join([
                                'Percentual Change: %{y}',
                                'Date: %{x}',
                                'Open: %{customdata[0]:.3f}',
                                'High: %{customdata[1]:.3f}',
                                'Low: %{customdata[2]:.3f}',
                                'Close: %{customdata[3]:.3f}'
                            ])))
                    
            else:
                
                for ind, value in enumerate(color_palete):

                    if i == ind: 
                        fig.add_trace(
                        go.Scatter(
                            x=df[v].index,
                            y=df[v],
                            mode="lines",
                            name = v,
                            line_color=value,
                            customdata=df_normal[['Open','High','Low','Close']],
                            hovertemplate='<br>'.join([
                                'Percentual Change: %{y}',
                                'Date: %{x}',
                                'Open: %{customdata[0]:.3f}',
                                'High: %{customdata[1]:.3f}',
                                'Low: %{customdata[2]:.3f}',
                                'Close: %{customdata[3]:.3f}'
                            ]),
                        )
                        )
        fig.update_layout(width=1300, height= 2000)
                        
    fig.update_layout(xaxis_rangeslider_visible=False)
    fig.update_layout(
        template="plotly_dark",
        plot_bgcolor = 'rgba(0, 0, 0, 0)',
        paper_bgcolor = 'rgba(0, 0, 0, 0)',
        font_color="white",
        font_size= 15
    )       
    fig.show()


    


In [154]:
candlestick(df_btc, 360)


In [155]:
candlestick(df_btc, 100, comparison = df_sol)

In [222]:
candlestick(df_btc, 100, indicators = [ 'bolinger bands','stdev' , 'stochastic', 'rsi', 'macd', 'adx'])

In [216]:
candlestick(df_btc, 100, comparison = df_sol, indicators = [ 'bolinger bands', 'rsi', 'stochastic', 'macd'])